# EKF short example 
From: https://docs.google.com/presentation/d/1fT3nua171rOZXKoyEC3YevsTVMiCbaPVEXpt2vZPRho/edit?usp=drive_link


<table><tr>
<td> <img src="data/ek1.png" alt="EKF short example 1" width="600"/> </td>
<td> <img src="data/ek2.png" alt="EKF short example 2" width="600"/> </td>
</tr></table>


<table><tr>
<td> <img src="data/ek3.png" alt="EKF short example 3" width="600"/> </td>
<td> <img src="data/ek4.png" alt="EKF short example 4" width="600"/> </td>
</tr></table>

In [12]:
import numpy as np 
from numpy.linalg import inv

X0_hat = np.array([[0, 5]]).T
P0_hat = np.diag([0.01, 1])
dt = 0.5
u0 = -2
S = 20
D = 40
F = np.identity(2)
F[0, 1] = dt
G = np.array([[0, 0.5]]).T
L = np.identity(2)
Q0 = 0.1 * np.identity(2)
R0 = 0.01
M = np.identity(1)
y1 = np.pi/6

In [15]:
def f(x, u):
    return F.dot(x) + G.dot(u)

# Measurement function h
def h(x, S, D):
    p = x[0, 0]
    return np.arctan2(S, D -p)

# Jacobian matrix H 
def HH(x, S, D):
    H = np.zeros((1, 2))
    p = x[0, 0]
    denom = (D - p)**2 + S**2
    H[0, 0] = S/denom
    return H

# Prediction step
X1_check = f(X0_hat, u0)
P1_check = np.dot(F, P0_hat).dot(F.T) + np.dot(L, Q0).dot(L.T)

# Measurement update step
H = HH(X1_check, S, D)

K = np.dot(P1_check, H.T).dot( inv(np.dot(H, P1_check).dot(H.T) + np.dot(M, R0).dot(M.T)) )
# print(K)

X1_hat = X1_check + K.dot(y1 - h(X1_check, S, D))
P1_hat = (np.identity(2) - np.dot(K, H)).dot(P1_check)

print(X1_hat)
print(P1_hat)

[[2.51335109]
 [4.01854318]]
[[0.35841804 0.49780283]
 [0.49780283 1.09694837]]
